## Create a BDBag with `participant.tsv` and `sample.tsv` in its data directory

In [34]:
import os
import bagit
from shutil import copy

In order to upload data to a FireCloud workspace we need two files, named `participant.tsv` and `sample.tsv`. The first holds a unique list of donor UUIDs, and the second a the same donor UUIDs with their corresponding specimen UUIDs. The values in these two columns of the table are the composite primary key. The third column holds cell-suspension UUIDs, followed by other columns which I copied from the manifest downloaded from the Explorer of the HCA browser.

In [36]:
os.mkdir('hca_manifest')  # the name of the BDBag
bag = bagit.make_bag('hca_manifest/', {'info': 'some info'})  # use bagit module to create a bag instance
files = list(filter(lambda x: x.endswith('.tsv'), os.listdir()))  # create list of files to copied into the data directory of the bag
for file in files:
    copy(file, 'hca_manifest/data')
['participant.tsv', 'sample.tsv'] == os.listdir('hca_manifest/data')  # sanity check

True

In [37]:
bag.save(manifests=True)  # save bag

In [38]:
bag.is_valid()  # sanity check

True